In [1]:
import numpy as np
import pandas as pd

import scanpy
import anndata as ad
import cloudpickle

import dask
import dask.array as da

from dask_ml.decomposition import IncrementalPCA
from multiprocessing.pool import ThreadPool
from scipy.sparse import csr_matrix
#from sparse import COO

In [2]:
dask.config.set(pool=ThreadPool(1))

data = ad.read_h5ad("/home/icb/jonas.flor/gastrulation_atlas/scvi/training/10k/2k_genes/integrated_adata.h5ad", backed='r').X
#batch_size = 40429
batch_size = 1000
def read_X(data, pos):
    return data[pos:pos+batch_size, :]

X = da.concatenate([da.from_delayed(dask.delayed(read_X)(data, pos), (batch_size, data.shape[1]), dtype='f4') for pos in range(0, data.shape[0], batch_size)])
del data
X = X.persist().rechunk((8192, -1))
X

dask.array<rechunk-merge, shape=(10000, 24552), dtype=float32, chunksize=(8192, 24552), chunktype=numpy.ndarray>

In [5]:
dask.config.set(pool=ThreadPool(30))

n_comps = 50
pca = IncrementalPCA(n_components=n_comps, iterated_power=3)
pca = da.compute(pca.fit_transform(X))
pca

/home/icb/jonas.flor/mambaforge/envs/atlas_pca/lib/python3.11/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


(array([[ 2.00739199e-03, -8.75690210e-04,  2.97290071e-03, ...,
         -1.14286043e-04,  2.90506045e-05,  1.73859341e-05],
        [ 4.48806320e-03, -2.61242456e-03,  3.10669930e-04, ...,
         -5.01194896e-05,  2.67211956e-05,  1.41542358e-05],
        [-5.93218004e-03,  3.03091301e-03,  3.00448308e-03, ...,
         -1.86394261e-04,  1.90238237e-04, -3.03349573e-04],
        ...,
        [ 1.35857672e-03, -2.80199990e-04, -1.37919438e-03, ...,
          3.08919148e-04,  3.51944166e-05,  7.56631675e-05],
        [ 4.26314595e-03, -9.24643301e-04,  1.37716058e-03, ...,
         -1.52951581e-05, -1.53554094e-05, -1.15242656e-06],
        [ 2.00037218e-03, -1.29600396e-03,  2.68972356e-03, ...,
          1.87723124e-05, -6.79489257e-05,  6.07014906e-07]]),)

In [5]:
pca[0].shape

(11441407, 50)

In [7]:
adata.obsm['X_pca'] = pca[0]

In [8]:
adata.write('/lustre/groups/ml01/workspace/monge_velo/data/mouse_gastrulation_atlas.h5ad')

OSError: Unable to open file (file is already open for read-only)